In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

import pyrosetta
from pyrosetta.rosetta.core.scoring import get_score_function
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover

pyrosetta.init("-mute core basic protocols -corrections::beta_nov16")

pickle_6YLA = '/home/zengyun1/AbRFC/data/test/CMAB0/6YLA_scores.pickle'

data = pd.read_pickle(pickle_6YLA)



┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python38.ubuntu 2024.42+release.3366cf78a3df04339d1982e94531b77b098ddb99 2024-10-11T08:24:04] retrieved from: http://www.pyrosetta.org


In [27]:
def calculate_neighbors(residue_number, pose_residue_index, inter_energy):
    neighbors_1 = []
    neighbors_2 = []

    for resid in range(residue_number):
        inter_energy_i = np.array(list(inter_energy[pose_residue_index, :][resid]))[1:]
        inter_energy_i = np.abs(inter_energy_i)

        if np.max(inter_energy_i) > 0.15:
            neighbors_2.append(resid)

        elif np.max(inter_energy_i) > 0.05:
            neighbors_1.append(resid)
    
    return neighbors_1, neighbors_2

In [30]:
def get_interface_data(inter_resid_data, pose_residue_index, neighbors_1, neighbors_2):
    attributes = [
        'complexed_sasa', 'dG', 'dSASA', 'dSASA_fraction', 'dSASA_sc',
        'dhSASA', 'dhSASA_rel_by_charge', 'dhSASA_sc', 'interface_residues'
    ]
    interface_data = {}
    
    indices = {'_0': pose_residue_index, '_1': neighbors_1, '_2': neighbors_2}

    for attr in attributes:
        data = np.nan_to_num(np.array(getattr(inter_resid_data, attr)), nan=0.0)
        for suffix, idx in indices.items():
            interface_data[f'i{attr}{suffix}'] = data[idx].sum()

    return interface_data


In [ ]:
def get_interface_data(inter_resid_data, pose_residue_index, neighbors_1, neighbors_2):


    attributes = [
        'complexed_sasa', 'dG', 'dSASA', 'dSASA_fraction', 'dSASA_sc',
        'dhSASA', 'dhSASA_rel_by_charge', 'dhSASA_sc', 'interface_residues'
    ]

    for attr in attributes:
        value = getattr(inter_resid_data, attr) 
        globals()['i' + attr] = np.nan_to_num(np.array(value), nan=0.0)
        
    # icomplexed_sasa = np.nan_to_num(np.array(inter_resid_data.complexed_sasa), nan=0.0)

    interface_data = {}

    keys_and_data = {'icomplexed_sasa': icomplexed_sasa,    'idG': idG,                 'idSASA': idSASA,
        'idSASA_fraction': idSASA_fraction,                 'idSASA_sc': idSASA_sc,     'idhSASA': idhSASA,
        'idhSASA_rel_by_charge': idhSASA_rel_by_charge,     'idhSASA_sc': idhSASA_sc,   'iinterface_residues': iinterface_residues}

    indices = {'_0': pose_residue_index,'_1': neighbors_1,'_2': neighbors_2}
    
    for key, data in keys_and_data.items():
        for suffix, idx in indices.items():
            interface_data[f'{key}{suffix}'] = data[idx].sum()

    return interface_data


In [2]:



pose_residue_index=100

pose = pyrosetta.pose_from_pdb('ref.pdb')


residue_number = pose.total_residue()


# Initialize score function and interface analyzer
sfxn = pyrosetta.create_score_function('beta_nov16')
sfxn(pose)

# Calculate inter energy and neighbors
inter_energy = pose.energies().residue_pair_energies_array()

pose_residue_index = pose_residue_index - 1

# neighbors_1, neighbors_2 = calculate_neighbors(residue_number, pose_residue_index, inter_energy)



In [ ]:
def calculate_interface_properties(pose,interface_chain,pose_residue_index,neighbors_1,neighbors_2):

    interface_analyzer = InterfaceAnalyzerMover()
    interface_analyzer.set_interface(interface_chain)
    interface_analyzer.set_calc_dSASA(True)
    interface_analyzer.set_compute_interface_energy(True)
    interface_analyzer.set_compute_packstat(True)
    interface_analyzer.apply(pose)

    inter_resid_data = interface_analyzer.get_all_per_residue_data()
    interface_data = get_interface_data(inter_resid_data, pose_residue_index, neighbors_1, neighbors_2)


    idelta_unsat_hbonds = interface_analyzer.get_all_data().delta_unsat_hbonds
    iinterface_hbonds = interface_analyzer.get_all_data().interface_hbonds
    ipackstat = interface_analyzer.get_all_data().packstat
    isc_value = interface_analyzer.get_all_data().sc_value
    
    interface_data['idelta_unsat_hbonds'] = idelta_unsat_hbonds
    interface_data['iinterface_hbonds'] = iinterface_hbonds
    interface_data['ipackstat'] = ipackstat
    interface_data['isc_value'] = isc_value


    return interface_data

210

In [ ]:
def calculate_residue_energy(pose, pose_residue_index, neighbors_1):
    resid_energy = pose.energies().residue_total_energies_array()


    energy_data = {}

    energy_data['total_score_0'] = resid_energy['total_score'][pose_residue_index].sum()
    energy_data['total_score_1'] = resid_energy['total_score'][neighbors_1].sum()

    energy_data['fa_atr_0'] = resid_energy['fa_atr'][pose_residue_index].sum()
    energy_data['fa_atr_1'] = resid_energy['fa_atr'][neighbors_1].sum()

    energy_data['fa_rep_0'] = resid_energy['fa_rep'][pose_residue_index].sum()
    energy_data['fa_rep_1'] = resid_energy['fa_rep'][neighbors_1].sum()


    return energy_data

In [ ]:
def extract_features(pose, pose_residue_index, interface_chain):
    
    residue_number = pose.total_residue()

    
    # Initialize score function and interface analyzer
    sfxn = pyrosetta.create_score_function('beta_nov16')
    sfxn(pose)
    
    interface_analyzer = InterfaceAnalyzerMover()
    interface_analyzer.set_interface(interface_chain)
    interface_analyzer.set_calc_dSASA(True)
    interface_analyzer.set_compute_interface_energy(True)
    interface_analyzer.set_compute_packstat(True)
    interface_analyzer.apply(pose)

    # Calculate inter energy and neighbors
    inter_energy = pose.energies().residue_pair_energies_array()

    pose_residue_index = pose_residue_index - 1

    neighbors_1, neighbors_2 = calculate_neighbors(residue_number, pose_residue_index, inter_energy)



    interface_analyzer = InterfaceAnalyzerMover()
    interface_analyzer.set_interface(interface_chain)
    interface_analyzer.set_calc_dSASA(True)
    interface_analyzer.set_compute_interface_energy(True)
    interface_analyzer.set_compute_packstat(True)
    interface_analyzer.apply(pose)
    
    idelta_unsat_hbonds = interface_analyzer.get_all_data().delta_unsat_hbonds
    iinterface_hbonds = interface_analyzer.get_all_data().interface_hbonds
    ipackstat = interface_analyzer.get_all_data().packstat
    isc_value = interface_analyzer.get_all_data().sc_value
    inter_resid_data = interface_analyzer.get_all_per_residue_data()
    
    # Get interface features
    interface_data = get_interface_data(inter_resid_data, pose_residue_index, neighbors_1, neighbors_2)

    # Get residue energy features
    energy_data = calculate_residue_energy(pose, pose_residue_index, neighbors_1)

    # Combine all the features into a single pd.Series
    new_feature = pd.Series({
        'dE2': pose.energies().total_energy(),
        'idelta_unsat_hbonds': interface_analyzer.get_all_data().delta_unsat_hbonds,
        'iinterface_hbonds': interface_analyzer.get_all_data().interface_hbonds,
        'ipackstat': interface_analyzer.get_all_data().packstat,
        'isc_value': interface_analyzer.get_all_data().sc_value,
        **interface_data,
        **energy_data
    })

    return new_feature

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_nov16.wts


NameError: name 'calculate_neighbors' is not defined

In [16]:
for i in neighbors_1:
    residue_index = i+1
    print(pose.pdb_info().chain(residue_index),pose.pdb_info().number(residue_index))

A 378
A 380
A 383


In [ ]:

def get_new_feature(pose,feature,pose_residue_index):
    sfxn = get_score_function(True)
    
    sfxn(pose)
    
    interface_analyzer = InterfaceAnalyzerMover()
    interface_analyzer.set_interface(interface_chain)
    interface_analyzer.set_calc_dSASA(True)  
    interface_analyzer.set_compute_interface_energy(True)  
    interface_analyzer.set_compute_packstat(True) 
    interface_analyzer.apply(pose)
    
    
    neighbors_1 = [] 
    neighbors_2 = [] 
    inter_energy = pose.energies().residue_pair_energies_array()
    resid_energy = pose.energies().residue_total_energies_array()

    pose_residue_index = pose_residue_index-1

    for resid in range(len(inter_energy[pose_residue_index,:])):
        inter_energy_i = np.array(list(inter_energy[pose_residue_index, :][resid]))[1:]
        if np.max(inter_energy_i) > 0.15 : neighbors_2.append(resid)
        elif np.max(inter_energy_i) > 0.05 : neighbors_1.append(resid)


    dE2 = pose.energies().total_energy()

    idelta_unsat_hbonds = interface_analyzer.get_all_data().delta_unsat_hbonds
    iinterface_hbonds = interface_analyzer.get_all_data().interface_hbonds
    ipackstat = interface_analyzer.get_all_data().packstat
    isc_value = interface_analyzer.get_all_data().sc_value

    icomplexed_sasa = interface_analyzer.get_all_per_residue_data().complexed_sasa
    icomplexed_sasa = np.array(icomplexed_sasa)
    icomplexed_sasa = np.nan_to_num(icomplexed_sasa, nan=0.0)
    icomplexed_sasa_0 = icomplexed_sasa[pose_residue_index].sum()
    icomplexed_sasa_1 = icomplexed_sasa[neighbors_1].sum()
    icomplexed_sasa_2 = icomplexed_sasa[neighbors_2].sum()

    idG = interface_analyzer.get_all_per_residue_data().dG
    idG = np.array(idG)
    idG = np.nan_to_num(idG, nan=0.0)
    idG_0 = idG[pose_residue_index].sum()
    idG_1 = idG[neighbors_1].sum()
    idG_2 = idG[neighbors_2].sum()

    idSASA = interface_analyzer.get_all_per_residue_data().dSASA
    idSASA = np.array(idSASA)
    idSASA = np.nan_to_num(idSASA, nan=0.0)
    idSASA_0 =idSASA[pose_residue_index].sum()
    idSASA_1 =idSASA[neighbors_1].sum()
    idSASA_2 =idSASA[neighbors_2].sum()

    idSASA_fraction = interface_analyzer.get_all_per_residue_data().dSASA_fraction
    idSASA_fraction = np.array(idSASA_fraction)
    idSASA_fraction = np.nan_to_num(idSASA_fraction, nan=0.0)
    idSASA_fraction_0 = idSASA_fraction[pose_residue_index].sum()
    idSASA_fraction_1 = idSASA_fraction[neighbors_1].sum()
    idSASA_fraction_2 = idSASA_fraction[neighbors_2].sum()

    idSASA_sc = interface_analyzer.get_all_per_residue_data().dSASA_sc
    idSASA_sc = np.array(idSASA_sc)
    idSASA_sc = np.nan_to_num(idSASA_sc, nan=0.0)
    idSASA_sc_0 = idSASA_sc[pose_residue_index].sum()
    idSASA_sc_1 = idSASA_sc[neighbors_1].sum()
    idSASA_sc_2 = idSASA_sc[neighbors_2].sum()

    idhSASA = interface_analyzer.get_all_per_residue_data().dhSASA
    idhSASA = np.array(idhSASA)
    idhSASA = np.nan_to_num(idhSASA, nan=0.0)
    idhSASA_0 = idhSASA[pose_residue_index].sum()
    idhSASA_1 = idhSASA[neighbors_1].sum()
    idhSASA_2 = idhSASA[neighbors_2].sum()

    idhSASA_rel_by_charge = interface_analyzer.get_all_per_residue_data().dhSASA_rel_by_charge
    idhSASA_rel_by_charge = np.array(idhSASA_rel_by_charge)
    idhSASA_rel_by_charge = np.nan_to_num(idhSASA_rel_by_charge, nan=0.0)
    idhSASA_rel_by_charge_0 = idhSASA_rel_by_charge[pose_residue_index].sum()
    idhSASA_rel_by_charge_1 = idhSASA_rel_by_charge[neighbors_1].sum()
    idhSASA_rel_by_charge_2 = idhSASA_rel_by_charge[neighbors_2].sum()

    idhSASA_sc = interface_analyzer.get_all_per_residue_data().dhSASA_sc
    idhSASA_sc = np.array(idhSASA_sc)
    idhSASA_sc = np.nan_to_num(idhSASA_sc, nan=0.0)
    idhSASA_sc_0 = idhSASA_sc[pose_residue_index].sum()
    idhSASA_sc_1 = idhSASA_sc[neighbors_1].sum()
    idhSASA_sc_2 = idhSASA_sc[neighbors_2].sum()

    iinterface_residues = interface_analyzer.get_all_per_residue_data().interface_residues
    iinterface_residues = np.array(iinterface_residues)
    iinterface_residues = np.nan_to_num(iinterface_residues, nan=0.0)
    iinterface_residues_0 = iinterface_residues[pose_residue_index].sum()
    iinterface_residues_1 = iinterface_residues[neighbors_1].sum()
    iinterface_residues_2 = iinterface_residues[neighbors_2].sum()

    total_score = resid_energy['total_score']
    total_score_0 = total_score[pose_residue_index].sum()
    total_score_1 = total_score[neighbors_1].sum()

    fa_atr = resid_energy['fa_atr']
    fa_atr_0 = fa_atr[pose_residue_index].sum()
    fa_atr_1 = fa_atr[neighbors_1].sum()

    fa_rep = resid_energy['fa_rep']
    fa_rep_0 = fa_rep[pose_residue_index].sum()
    fa_rep_1 = fa_rep[neighbors_1].sum()

    fa_sol = resid_energy['fa_sol']
    fa_sol_0 = fa_sol[pose_residue_index].sum()
    fa_sol_1 = fa_sol[neighbors_1].sum()

    fa_intra_rep = resid_energy['fa_intra_rep']
    fa_intra_rep_0 = fa_intra_rep[pose_residue_index].sum()
    fa_intra_rep_1 = fa_intra_rep[neighbors_1].sum()

    fa_intra_sol_xover4 = resid_energy['fa_intra_sol_xover4']
    fa_intra_sol_xover4_0 = fa_intra_sol_xover4[pose_residue_index].sum()
    fa_intra_sol_xover4_1 = fa_intra_sol_xover4[neighbors_1].sum()

    lk_ball_wtd = resid_energy['lk_ball_wtd']
    lk_ball_wtd_0 = lk_ball_wtd[pose_residue_index].sum()
    lk_ball_wtd_1 = lk_ball_wtd[neighbors_1].sum()

    fa_elec = resid_energy['fa_elec']
    fa_elec_0 = fa_elec[pose_residue_index].sum()
    fa_elec_1 = fa_elec[neighbors_1].sum()

    pro_close = resid_energy['pro_close']
    pro_close_0 = pro_close[pose_residue_index].sum()
    pro_close_1 = pro_close[neighbors_1].sum()

    hbond_sr_bb = resid_energy['hbond_sr_bb']
    hbond_sr_bb_0 = hbond_sr_bb[pose_residue_index].sum()
    hbond_sr_bb_1 = hbond_sr_bb[neighbors_1].sum()

    hbond_lr_bb = resid_energy['hbond_lr_bb']
    hbond_lr_bb_0 = hbond_lr_bb[pose_residue_index].sum()
    hbond_lr_bb_1 = hbond_lr_bb[neighbors_1].sum()

    hbond_bb_sc = resid_energy['hbond_bb_sc']
    hbond_bb_sc_0 = hbond_bb_sc[pose_residue_index].sum()
    hbond_bb_sc_1 = hbond_bb_sc[neighbors_1].sum()

    hbond_sc = resid_energy['hbond_sc']
    hbond_sc_0 = hbond_sc[pose_residue_index].sum()
    hbond_sc_1 = hbond_sc[neighbors_1].sum()

    
    new_feature = pd.Series({
    'dE2' : dE2,
    'idelta_unsat_hbonds' : idelta_unsat_hbonds,
    'iinterface_hbonds' : iinterface_hbonds,
    'ipackstat' : ipackstat,
    'isc_value' : isc_value,
    'icomplexed_sasa_0' : icomplexed_sasa_0,
    'icomplexed_sasa_1' : icomplexed_sasa_1,
    'icomplexed_sasa_2' : icomplexed_sasa_2,
    'idG_0' : idG_0,
    'idG_1' : idG_1,
    'idG_2' : idG_2,
    'idSASA_0' : idSASA_0,
    'idSASA_1' : idSASA_1,
    'idSASA_2' : idSASA_2,
    'idSASA_fraction_0' : idSASA_fraction_0,
    'idSASA_fraction_1' : idSASA_fraction_1,
    'idSASA_fraction_2' : idSASA_fraction_2,
    'idSASA_sc_0' : idSASA_sc_0,
    'idSASA_sc_1' : idSASA_sc_1,
    'idSASA_sc_2' : idSASA_sc_2,
    'idhSASA_0' : idhSASA_0,
    'idhSASA_1' : idhSASA_1,
    'idhSASA_2' : idhSASA_2,
    'idhSASA_rel_by_charge_0' : idhSASA_rel_by_charge_0,
    'idhSASA_rel_by_charge_1' : idhSASA_rel_by_charge_1,
    'idhSASA_rel_by_charge_2' : idhSASA_rel_by_charge_2,
    'idhSASA_sc_0' : idhSASA_sc_0,
    'idhSASA_sc_1' : idhSASA_sc_1,
    'idhSASA_sc_2' : idhSASA_sc_2,
    'iinterface_residues_0' : iinterface_residues_0,
    'iinterface_residues_1' : iinterface_residues_1,
    'iinterface_residues_2' : iinterface_residues_2,
    'total_score_0' : total_score_0,
    'total_score_1' : total_score_1,
    'fa_atr_0' : fa_atr_0,
    'fa_atr_1' : fa_atr_1,
    'fa_rep_0' : fa_rep_0,
    'fa_rep_1' : fa_rep_1,
    'fa_sol_0' : fa_sol_0,
    'fa_sol_1' : fa_sol_1,
    'fa_intra_rep_0' : fa_intra_rep_0,
    'fa_intra_rep_1' : fa_intra_rep_1,
    'fa_intra_sol_xover4_0' : fa_intra_sol_xover4_0,
    'fa_intra_sol_xover4_1' : fa_intra_sol_xover4_1,
    'lk_ball_wtd_0' : lk_ball_wtd_0,
    'lk_ball_wtd_1' : lk_ball_wtd_1,
    'fa_elec_0' : fa_elec_0,
    'fa_elec_1' : fa_elec_1,
    'pro_close_0' : pro_close_0,
    'pro_close_1' : pro_close_1,
    'hbond_sr_bb_0' : hbond_sr_bb_0,
    'hbond_sr_bb_1' : hbond_sr_bb_1,
    'hbond_lr_bb_0' : hbond_lr_bb_0,
    'hbond_lr_bb_1' : hbond_lr_bb_1,
    'hbond_bb_sc_0' : hbond_bb_sc_0,
    'hbond_bb_sc_1' : hbond_bb_sc_1,
    'hbond_sc_0' : hbond_sc_0,
    'hbond_sc_1' : hbond_sc_1})

    return new_feature

In [3]:
new_feature1_columns = ['dE2',
 'idelta_unsat_hbonds',
 'iinterface_hbonds',
 'ipackstat',
 'isc_value',
 'icomplexed_sasa_0',
 'icomplexed_sasa_1',
 'icomplexed_sasa_2',
 'idG_0',
 'idG_1',
 'idG_2',
 'idSASA_0',
 'idSASA_1',
 'idSASA_2',
 'idSASA_fraction_0',
 'idSASA_fraction_1',
 'idSASA_fraction_2',
 'idSASA_sc_0',
 'idSASA_sc_1',
 'idSASA_sc_2',
 'idhSASA_0',
 'idhSASA_1',
 'idhSASA_2',
 'idhSASA_rel_by_charge_0',
 'idhSASA_rel_by_charge_1',
 'idhSASA_rel_by_charge_2',
 'idhSASA_sc_0',
 'idhSASA_sc_1',
 'idhSASA_sc_2',
 'iinterface_residues_0',
 'iinterface_residues_1',
 'iinterface_residues_2',
 'total_score_0',
 'total_score_1',
 'fa_atr_0',
 'fa_atr_1',
 'fa_rep_0',
 'fa_rep_1',
 'fa_sol_0',
 'fa_sol_1',
 'fa_intra_rep_0',
 'fa_intra_rep_1',
 'fa_intra_sol_xover4_0',
 'fa_intra_sol_xover4_1',
 'lk_ball_wtd_0',
 'lk_ball_wtd_1',
 'fa_elec_0',
 'fa_elec_1',
 'pro_close_0',
 'pro_close_1',
 'hbond_sr_bb_0',
 'hbond_sr_bb_1',
 'hbond_lr_bb_0',
 'hbond_lr_bb_1',
 'hbond_bb_sc_0',
 'hbond_bb_sc_1',
 'hbond_sc_0',
 'hbond_sc_1']

In [8]:
total_feature=[]

for index, feature in tqdm(data.iterrows(), total=len(data)):


    try:
        chain_id = feature['chain']
        origin_aa = feature['refAA']
        target_aa = feature['mutAA']
        pack_radius = 5.0
        
        interface_chain = feature['sides'][0]+"_"+feature['sides'][1]
    
        pose = pyrosetta.pose_from_pdb('ref.pdb')

        if feature['res'][-1].isalpha():
            numeric_part = feature['res'][:-1]
            letter_part = feature['res'][-1]
            pose_residue_index = pose.pdb_info().pdb2pose(chain_id, res=int(numeric_part), icode=letter_part)
        else:
            pose_residue_index = pose.pdb_info().pdb2pose(chain_id, res=int(feature['res']))
    
    
        
        mut_pose = pyrosetta.pose_from_pdb('ref.pdb')
        mutants.mutate_residue(mut_pose, pose_residue_index, target_aa,pack_radius=pack_radius)
        # mut_pose.dump_pdb(mut_protein)
        
        ref_pose = pyrosetta.pose_from_pdb('ref.pdb')
        mutants.mutate_residue(ref_pose, pose_residue_index,origin_aa, pack_radius=pack_radius) #不突变
        
        new_ref_feature = get_new_feature(ref_pose,feature,pose_residue_index)
        new_mut_feature = get_new_feature(mut_pose,feature,pose_residue_index)
        new_feature1 = new_ref_feature - new_mut_feature
    except Exception as e:
        print(f"Error processing index {index}: {e}")
        new_feature1 = pd.Series({key: "NaN" for key in new_feature1_columns})

    
    new_feature2 = pd.Series({
    'pdb_ref' : feature['pdb_ref'],
    'chain' : feature['chain'],
    'res' : feature['res'],
    'refAA' : feature['refAA'],
    'mutAA' : feature['mutAA'],
    'sides' : feature['sides'],
    'pdb_ref_repack' : feature['pdb_ref_repack'],
    'pdb_mut_repack' : feature['pdb_mut_repack'],
    'aif_score' : feature['aif_score'],
    'sin_if' : feature['sin_if'],
    'sin_res' : feature['sin_res'],
    'sin_norm' : feature['sin_norm'],
    'yclf' : feature['yclf'],
    'yreg' : feature['yreg'],
    'label' : feature['label']})
    
    new_feature = pd.concat([new_feature1, new_feature2])
    total_feature.append(new_feature)

    
total_feature = pd.concat(total_feature, axis=1).transpose()

total_feature.to_pickle("final_feature_df.pkl")


  0%|          | 0/954 [00:00<?, ?it/s]

Error processing index 0: name 'mutants' is not defined


NameError: name 'new_feature1_columns' is not defined